# Mise en forme des données et intégration dans une base de donnée

In [1]:
import pandas as pd

## 1. Charger et examiner le fichier

In [2]:
path_to_file = './data/darkness-dataset.csv'

with open(path_to_file, 'r', encoding='utf-8') as f:
    raw_lines = [next(f) for _ in range(3)]

for raw_line in raw_lines:
    print(raw_line)

id,name,country_code,position,city,current_company,about,posts,experience,url,people_also_viewed,educations_details,education,recommendations_count,avatar,languages,certifications,recommendations,volunteer_experience,courses,followers,connections,current_company_company_id,current_company_name,publications,patents,projects,organizations,input_url,linkedin_id;;;;

"random-id-2468,""Random Name"",""KW"","""",""Kuwait"",""{""""company_id"""":""""-seara"""",""""industry"""":""""Seara Alimentos LTDA"""",""""link"""":""""https://br.linkedin.com/company/-seara?trk=public_profile_topcard-current-company"""",""""name"""":""""Random Alimentos LTDA""""}"",""Experienced Unit Sales Manager with a demonstrated history of working in the foodâ€¦"",""[{""""attribution"""":""""By Random Name"""",""""created_at"""":""""2018-09-05T00:00:00.000Z"""",""""img"""":""""https://static.licdn.com/scds/common/u/img/pic/pic_pulse_stock_article_1.jpg"""",""""interaction"""":""""By Random Name"""",""""link"""":""""ht

## 2. Nettoyage des données

In [3]:
path_to_file = './data/darkness-dataset.csv'

#Lecture ligne par ligne du fichier
with open(path_to_file, 'r', encoding='utf-8-sig') as file:
    for line_number, line in enumerate(file, start=1):
        
        if line_number == 1:
            # Récupération des noms de colonnes sur la premiere ligne
            columns = line.strip().split(",")
            columns = [col.replace('linkedin_id;;;;', 'linkedin_id') for col in columns]
            #print(columns)
            df = pd.DataFrame(columns=columns)

        else :
        #elif line_number == 8 : # Pour débugger ligne par ligne
            
            # Paramètres
            temp_ls = []
            temp_dict = {}
            temp_list_of_dicts = []
            temp_string = ""
                        
            in_dict = False  
            in_list = False
            in_string = False  
            start_key = False
            key_string = ""
            
            cp = 0
            
            line_split = line.strip().split(",")
            for elem in line_split:
                elem = elem.replace('"', '').strip()
                #print("START")
                #print(line_number)
                #print(in_dict, in_list, in_string)
                #print(elem)

                # None pour les elements vides
                if elem == "":
                    temp_ls.append(None)
                    
                # Pour la colonne city où c'est une liste 
                elif cp == 4 :
                    in_string = True
                    temp_string = elem
                    
                elif in_string and (not elem.startswith("{")) and (not ":" in elem):
                    temp_string += ", "+elem
                    
                # cas particuler de "location:"" : clé dont la valeur est un string avec des virgules
                elif "location:" in elem:
                    in_string = True
                    temp_string = elem.split(":", 1)[1].strip()  # Récupère la valeur initiale de la localisation
                    
                                    
                # Début d'une liste de dictionnaires
                elif elem.startswith("[{"):
                    in_list = True
                    in_dict = True
                    temp_list_of_dicts = []
                    temp_dict = {}
                    
                    # Cas particulier de la clé description
                    if "description:" in elem :
                        start_key = True
                        in_string = True
                        in_dict = True
                        elem_key, elem_value = elem[2:].split(":", 1)
                        temp_dict[elem_key.strip()] = ""
                        key_string = elem_key.strip()
                        temp_string = elem_value.strip()
                        
                    # Cas particulier des listes de dictionnaires vides
                    elif elem == "[{}" :
                        elem_key, elem_value = elem[1:]
                        temp_dict[elem_key.strip()] = elem_value.strip()
                        
                    else : 
                        elem_key, elem_value = elem[2:].split(":", 1)
                        temp_dict[elem_key.strip()] = elem_value.strip()
                    
                
                # Debut dictionnaire sans liste
                elif elem.startswith("{") and not in_list:
                    in_dict = True
                    temp_dict = {}
                    
                    first_entry = elem[1:]
                    if ":" in first_entry:
                        elem_key, elem_value = first_entry.split(":", 1)
                        temp_dict[elem_key.strip()] = elem_value.strip()
                        
                    if in_string :
                        temp_ls.append(temp_string)
                        temp_string = ""
                        in_string = False
                        
                # Fin d'un dictionnaire
                elif in_dict and elem.endswith("}"):
                    last_entry = elem[:-1]
                    if ":" in last_entry:
                        elem_key, elem_value = last_entry.split(":", 1)
                        temp_dict[elem_key.strip()] = elem_value.strip()
                    
                    if in_list:
                        temp_list_of_dicts.append(temp_dict)
                        in_dict = False
                    else:
                        temp_ls.append(temp_dict)
                        in_dict = False 
                
                # Remplissage d'un dico
                elif in_list and ":" in elem and not elem.endswith("}]"):
                    elem_key, elem_value = elem.split(":", 1)
                    if elem_key == "field" :
                        start_key = True
                        in_string = True
                        in_dict = True
                        temp_dict[elem_key.strip()] = ""
                        key_string = elem_key.strip()
                        temp_string = elem_value.strip()
                    else :
                        temp_dict[elem_key.strip()] = elem_value.strip()

                # Fin de la liste de dictionnaires
                elif elem.endswith("}]") and in_list:
                    last_entry = elem[:-2]
                    if ":" in last_entry:
                        elem_key, elem_value = last_entry.split(":", 1)
                        temp_dict[elem_key.strip()] = elem_value.strip()
                    
                    # Ajouter le dernier dictionnaire à la liste, puis la liste complète
                    temp_list_of_dicts.append(temp_dict)
                    temp_ls.append(temp_list_of_dicts)
                    in_list = False
                    in_dict = False
                    in_string = False
                    
                # Cas particulier : Fin de dictionnaire sans }
                elif in_list and not elem.startswith("{") and (":" not in elem or "}" not in elem):
                    temp_list_of_dicts.append(temp_dict)  
                    temp_ls.append(temp_list_of_dicts) 
                    in_list = False
                    in_dict = False
                
                elif in_dict:
                    if ":" in elem:
                        elem_key, elem_value = elem.split(":", 1)
                        temp_dict[elem_key.strip()] = elem_value.strip()
                    elif start_key :
                        temp_dict[key_string] = temp_string
                        key_string = ""
                        temp_string = ""
                        start_key = False
                        in_string = False
                        in_dict = False
                        
                elif cp > 4 :
                    if isinstance(elem, str) and temp_string == "":
                        in_string = True
                        temp_string = elem
                    elif isinstance(elem, str) & in_string == True :
                        temp_string += elem
                    elif not(isinstance(elem, str)) & in_string == True :
                        temp_ls.append(temp_string)
                        in_string = False
                        temp_string = ""
                
                # Pour les elements simples
                else:
                    temp_ls.append(elem)
                    
                # Debug
                #if len(temp_ls) > 1:
                    #print("------------------------------------------------------------------------------------------------------------")
                    #for temp_elem in temp_ls:
                        #print(temp_elem)
                        #print()
                        
                cp += 1

            if len(temp_ls) < len(df.columns):
                temp_ls.extend([None] * (len(df.columns) - len(temp_ls)))

            # Ajouter la ligne modifiée au DataFrame
            df.loc[len(df)] = temp_ls

        # Condition d'arrêt pour la démonstration (on arrête après la 2ème ligne)
        #if line_number > 10:
            #break
        
# Afficher le DataFrame pour vérifier le contenu
df

,id,name,country_code,position,city,current_company,about,posts,experience,url,...,followers,connections,current_company_company_id,current_company_name,publications,patents,projects,organizations,input_url,linkedin_id
0,random-id-2468,Random Name,KW,None,Kuwait,"{'company_id': '-seara', 'industry': 'Seara Al...","[{'attribution': 'By Random Name', 'created_at...",None,Experienced Unit Sales Manager with a demonstr...,[{'profile_link': 'https://kw.linkedin.com/in/...,...,None,None,None,None,None,None,None,None,None,None
1,random-id-1357,Random Name,IN,Random EduTech,"Noida, Uttar Pradesh, India","{'company_id': 'byjus', 'industry': 'BYJU'S', ...",None,None,"[{'company': 'Random EduTech', 'company_id': '...","New Delhi Area, India",...,None,None,None,"[{'end_date': 'Present', 'membership_number': ...","112, byjus, Random EduTech",None,None,None,None,None
2,random-id-8642,Random Name,None,None,None,"{'company_id': 'shepard-exposition-services', ...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,random-id-9512,Random Name,UK,Design Engineer (System health) at Random Powe...,Galashiels,"{'company_id': 'edf-energy', 'industry': 'EDF ...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,random-id-8642,Random Name,KW,None,Kuwait,"{'company_id': 'global-infinity', 'industry': ...",None,None,[{'profile_link': 'https://kw.linkedin.com/in/...,Experienced Chief Executive Officer with a dem...,...,None,None,None,None,None,None,None,None,None,None
5,random-id-7531,Random Name,KW,None,Kuwait,"{'company_id': 'global-infinity', 'industry': ...",None,None,[{'profile_link': 'https://kw.linkedin.com/in/...,Experienced Chief Executive Officer with a dem...,...,None,None,None,None,None,None,None,None,None,None
6,random-id-7890,Random Name,DZ,Logging Operations & Petrophysical QA/QC super...,"Biskra, Algeria","{'industry': 'None at present', 'name': 'None ...",None,"[{'company': 'None at present', 'description':...",SAHARA /Hassi Messaoud,[{'profile_link': 'https://dz.linkedin.com/in/...,...,None,None,"[{'membership_number': 'null', 'membership_typ...","196, 193, None at present",None,None,None,None,None,None
7,random-id-4820,Random Name,NG,Volkswagen Product at Random Auto Sales,Nigeria,"{'industry': 'CollinsCAR-Automobile_inv.org', ...","[{'attribution': 'By Random Name', 'created_at...","[{'company': 'Random Auto Sales', 'description...",We encourage you to browse our online inventor...,[{'profile_link': 'https://ng.linkedin.com/in/...,...,None,None,None,None,None,None,None,None,None,None
8,random-id-9025,Random Name,UK,None,None,"{'company_id': 'oldmill', 'industry': 'Old Mil...",None,I have worked in Financial Services since late...,None,None,...,None,None,None,None,None,None,None,None,None,None
9,random-id-5837,Random Name,EG,Food Safety & hygiene coordinator at Random Ho...,Cairo,"{'company_id': 'steigenberger', 'industry': 'S...",None,"[{'description': '', 'description_html': 'null...","Ø¬Ù†ÙˆØ¨ Ø³ÙŠÙ†Ø§Ø¡ Ø´Ø±Ù… Ø§Ù„Ø´ÙŠØ® Ù…ØµØ±, ...","[{'degree': 'Bachelor's degree', 'description'...",...,None,None,None,None,None,None,None,None,None,None


## 3. Sauvegarde des données

In [4]:
df.to_excel("./data/cleaned_data.xlsx", index=False)